In [1]:
# For reading credentials from the .env file
import os
from dotenv import load_dotenv
import pandas as pd

from langchain.document_loaders import PyPDFLoader, DataFrameLoader
from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter

# WML python SDK
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

# Load API credentials from .env file
load_dotenv()
try:
    API_KEY = os.environ.get("API_KEY")
    project_id = os.environ.get("PROJECT_ID")
except KeyError:
    API_KEY = input("Please enter your WML api key (hit enter): ")
    project_id = input("Please enter your project_id (hit enter): ")

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": API_KEY
}


def get_model(model_type, max_tokens, min_tokens, decoding, temperature):

    generate_params = {
        GenParams.MAX_NEW_TOKENS: max_tokens,
        GenParams.MIN_NEW_TOKENS: min_tokens,
        GenParams.DECODING_METHOD: decoding,
        GenParams.TEMPERATURE: temperature
    }

    model = Model(
        model_id=model_type,
        params=generate_params,
        credentials=credentials,
        project_id=project_id
    )

    return model


def get_lang_chain_model(model_type, max_tokens, min_tokens, decoding, temperature):

    base_model = get_model(model_type, max_tokens, min_tokens, decoding, temperature)
    langchain_model = WatsonxLLM(model=base_model)

    return langchain_model

We import the HuggingFaceEmbeddings class from langchain_community.embeddings.
We define the model name (you can choose a different pre-trained model from the Hugging Face library).
We create an instance of HuggingFaceEmbeddings with the chosen model name.
We define the text to embed ("hello world").
We use the embed_query method of the hf_embeddings object to get the embedding for the text. This method returns a list of floats representing the encoded text.
Finally, we print the embedding list.
Note:

This example uses embed_query for a single sentence. If you have multiple sentences to embed, use embed_documents.
The actual values in the embedding list will depend on the specific model you choose and the text you embed.

In [2]:
def answer_questions_from_dataframe(file_path, question):
    # Specify model parameters
    model_type = "meta-llama/llama-2-70b-chat"
    max_tokens = 300
    min_tokens = 100
    decoding = DecodingMethods.GREEDY
    temperature = 0.7

    # Get the watsonx model that can be used with LangChain
    model = get_lang_chain_model(model_type, max_tokens, min_tokens, decoding, temperature)

    # Load the dataframe from the pickle file
    df = pd.read_pickle(file_path)
    # Get the text content from the "Content" column
    text_content = df["Content"].tolist()

    from langchain_community.embeddings import HuggingFaceEmbeddings
    import numpy as np  # Optional, for numerical operations on embeddings (if needed)

    # Choose an appropriate model:
    model_name = "sentence-transformers/all-mpnet-base-v2"  # Replace with your desired model if needed
    # Set device (CPU or GPU) based on your hardware and performance requirements:
    model_kwargs = {'device': 'cpu'}  # Change to 'cuda' for GPU usage (if available)
    # Encoding options (normalization is often recommended):
    encode_kwargs = {'normalize_embeddings': True}  # Experiment with normalization

    # Initialize the embedding model:
    hf = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

    # Create an empty list to store the embeddings:
    embeddings_list = []

    for text in text_content:
        # Preprocess text if necessary (e.g., tokenization, cleaning)
        # processed_text = preprocess_text(text)  # Implement your preprocessing function (optional)

        # **Corrected embedding generation:**
        embedding = hf.embed_query(text)  # Use embed_query for single text

        # Append the embedding vector directly (no need to extract):
        embeddings_list.append(embedding)

    # Now you have a list of embeddings (each embedding is a list of floats)
    # Use these embeddings for your further analysis or tasks
    #print(embeddings_list)
    loaders = [DataFrameLoader(df, page_content_column='Content')]
    index = VectorstoreIndexCreator(
        embedding=HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
        ),
        text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    ).from_loaders(loaders)

    chain = RetrievalQA.from_chain_type(
        llm=model,
        chain_type="stuff",
        retriever=index.vectorstore.as_retriever(),
        input_key="question"
    )

    response_text = chain.run(question)

    print("--------------------------------- Generated response -----------------------------------")
    print(response_text)
    print("*********************************************************************************************")

    return response_text



In [3]:
#help(DataFrameLoader)

In [4]:
def main():
    # Test answering questions based on the provided dataframe
    question = "What are the limitations of generative AI models?"
    
    # Provide the path relative to the dir in which the script is running
    file_path = "../data/output.pkl"

    answer_questions_from_dataframe(file_path, question)

In [5]:
#help(hf)

In [6]:
# Invoke the main function
if __name__ == "__main__":
    main()


c:\Users\066226758\workspace\workshop\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\066226758\workspace\workshop\.venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


--------------------------------- Generated response -----------------------------------
 I don't know.

The provided code does not relate to generative AI models or their limitations. It appears to be a Python script that searches for files in a directory and saves their names to a text file. Therefore, I cannot provide an answer to the question.

Do you have any other questions or need assistance with a different topic? I'm here to help!



























































































































































































































*********************************************************************************************
